In [ ]:
%%sh
pip install -q sagemaker --upgrade

In [2]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'huggingface/demo-customer-reviews'

2.46.0


In [3]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.23-1',
                                     role=sagemaker.get_execution_role(),
                                     instance_type='ml.m5.2xlarge',
                                     instance_count=1)

In [4]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code='preprocessing.py',
    
    outputs=[
        ProcessingOutput(
                output_name='training',
                source='/opt/ml/processing/output/training'),
        ProcessingOutput(
                output_name='validation',
                source='/opt/ml/processing/output/validation')
    ],
    
    arguments=[
               '--threshold', '4',
               '--s3-bucket', bucket,
               '--s3-prefix', prefix
    ]
)


Job Name:  sagemaker-scikit-learn-2021-06-16-07-57-40-837
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-16-07-57-40-837/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'training', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-16-07-57-40-837/output/training', 'LocalPath': '/opt/ml/processing/output/training', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'validation', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-16-07-57-40-837/output/validation', 'LocalPath': '/opt/ml/processing/output/validation', 'S3UploadMode': 'EndOfJob'}}]
........................Collecting transformers>=4.4.

In [5]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])

s3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-16-07-57-40-837/output/training
s3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-16-07-57-40-837/output/validation
